In [1]:
#Import Libraries and Load Data
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.chart import PieChart, LineChart, BarChart, ScatterChart, Reference

In [7]:
df=pd.read_excel(r"C:\Users\Adithya Ajith\Downloads\Sample Superstore.xlsx")
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [9]:
#Raw Data Sheet
wb = Workbook()
ws_raw = wb.active
ws_raw.title = "Raw Data"
for row in dataframe_to_rows(df, index=False, header=True):
    ws_raw.append(row)

In [11]:
#Dashboard Sheet Setup
ws_dash = wb.create_sheet("Dashboard")
ws_dash['A1'] = "Superstore Marketing Insights Dashboard"
ws_dash['A1'].font = Font(size=16, bold=True)
ws_dash['A1'].alignment = Alignment(horizontal='center')
ws_dash.column_dimensions['A'].width = 20
ws_dash.column_dimensions['B'].width = 15
ws_dash.column_dimensions['C'].width = 15

In [13]:
# Header style
header_font = Font(bold=True)
header_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')

In [15]:
#Category Sales Summary
category_df = df.groupby('Category')[['Sales']].sum().reset_index()
category_df['Sales'] = category_df['Sales'].round(2)
category_df['% of Total'] = (category_df['Sales'] / category_df['Sales'].sum() * 100).round(2)
ws_dash['A3'] = "Sales by Category"
ws_dash['A3'].font = Font(size=12, bold=True)
for i, row in enumerate(dataframe_to_rows(category_df, index=False, header=True), start=4):
    for j, value in enumerate(row, start=1):
        cell = ws_dash.cell(row=i, column=j, value=value)
        if i == 4:  # Header row
            cell.font = header_font
            cell.fill = header_fill

In [17]:
#Monthly Sales Trend
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m/%d/%Y')
monthly_df = df.groupby(df['Order Date'].dt.to_period('M'))[['Sales']].sum().reset_index()
monthly_df['Order Date'] = monthly_df['Order Date'].astype(str)
monthly_df['Sales'] = monthly_df['Sales'].round(2)
ws_dash['A10'] = "Monthly Sales Trend"
ws_dash['A10'].font = Font(size=12, bold=True)
for i, row in enumerate(dataframe_to_rows(monthly_df, index=False, header=True), start=11):
    for j, value in enumerate(row, start=1):
        cell = ws_dash.cell(row=i, column=j, value=value)
        if i == 11:  # Header row
            cell.font = header_font
            cell.fill = header_fill

In [19]:
#Regional Sales Summary
region_df = df.groupby('Region')[['Sales']].sum().reset_index()
region_df['Sales'] = region_df['Sales'].round(2)
ws_dash['A20'] = "Sales by Region"
ws_dash['A20'].font = Font(size=12, bold=True)
for i, row in enumerate(dataframe_to_rows(region_df, index=False, header=True), start=21):
    for j, value in enumerate(row, start=1):
        cell = ws_dash.cell(row=i, column=j, value=value)
        if i == 21:  # Header row
            cell.font = header_font
            cell.fill = header_fill
        if i == 22 and region_df['Sales'].max() == value:  # Highlight top region
            cell.fill = PatternFill(start_color='00FF00', fill_type='solid')

In [21]:
#Profit vs. Discount Analysis
discount_df = df.groupby('Discount')[['Sales', 'Profit']].sum().reset_index()
discount_df['Sales'] = discount_df['Sales'].round(2)
discount_df['Profit'] = discount_df['Profit'].round(2)
ws_dash['A30'] = "Profit vs. Discount Impact"
ws_dash['A30'].font = Font(size=12, bold=True)
for i, row in enumerate(dataframe_to_rows(discount_df, index=False, header=True), start=31):
    for j, value in enumerate(row, start=1):
        cell = ws_dash.cell(row=i, column=j, value=value)
        if i == 31:  # Header row
            cell.font = header_font
            cell.fill = header_fill

In [23]:
#Top 5 Products by Sales
top_products_df = df.groupby('Product Name')[['Sales']].sum().nlargest(5, 'Sales').reset_index()
top_products_df['Sales'] = top_products_df['Sales'].round(2)
ws_dash['A40'] = "Top 5 Products by Sales"
ws_dash['A40'].font = Font(size=12, bold=True)
for i, row in enumerate(dataframe_to_rows(top_products_df, index=False, header=True), start=41):
    for j, value in enumerate(row, start=1):
        cell = ws_dash.cell(row=i, column=j, value=value)
        if i == 41:  # Header row
            cell.font = header_font
            cell.fill = header_fill

Visualizations

In [28]:
# Pie Chart: Category Sales
pie = PieChart()
pie.title = "Sales Distribution by Category"
data = Reference(ws_dash, min_col=2, min_row=5, max_row=7)
labels = Reference(ws_dash, min_col=1, min_row=5, max_row=7)
pie.add_data(data, titles_from_data=True)
pie.set_categories(labels)
pie.width = 10
pie.height = 7
ws_dash.add_chart(pie, "E5")

In [30]:
# Line Chart: Monthly Sales
line = LineChart()
line.title = "Monthly Sales Trend (2014-2017)"
line.x_axis.title = "Month"
line.y_axis.title = "Sales ($)"
data = Reference(ws_dash, min_col=2, min_row=12, max_row=12 + len(monthly_df) - 1)
labels = Reference(ws_dash, min_col=1, min_row=12, max_row=12 + len(monthly_df) - 1)
line.add_data(data, titles_from_data=True)
line.set_categories(labels)
line.width = 15
line.height = 7
ws_dash.add_chart(line, "E20")

In [32]:
# Bar Chart: Regional Sales
bar = BarChart()
bar.title = "Sales by Region"
bar.x_axis.title = "Region"
bar.y_axis.title = "Sales ($)"
data = Reference(ws_dash, min_col=2, min_row=22, max_row=25)
labels = Reference(ws_dash, min_col=1, min_row=22, max_row=25)
bar.add_data(data, titles_from_data=True)
bar.set_categories(labels)
bar.width = 10
bar.height = 7
ws_dash.add_chart(bar, "E35")

In [42]:
# Scatter Chart: Profit vs. Discount
scatter = ScatterChart()
scatter.title = "Profit vs. Discount Impact"
scatter.x_axis.title = "Discount (%)"
scatter.y_axis.title = "Profit ($)"
xvalues = Reference(ws_dash, min_col=1, min_row=32, max_row=32 + len(discount_df) - 1)  # Discount
yvalues = Reference(ws_dash, min_col=3, min_row=32, max_row=32 + len(discount_df) - 1)  # Profit
scatter.add_data(yvalues, titles_from_data=True)  # Add Profit as the y-values
scatter.set_categories(xvalues)  # Set Discount as the x-values
scatter.width = 10
scatter.height = 7
ws_dash.add_chart(scatter, "E50")

In [47]:
wb.save('superstore_dashboard_portfolio.xlsx')